In [1]:
import pandas as pd
import numpy as np
import spacy as sp
from imdb import Cinemagoer
import matplotlib.pyplot as plt
import datetime
from collections import Counter
import warnings

warnings.filterwarnings("ignore")

from cleaning_funcs import *

ia = Cinemagoer()

%matplotlib inline

#IMDb Docs: https://imdbpy.readthedocs.io/en/latest/index.html

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
data = load_data('movie_list_current.csv')
data = data[['Month', 'Pick', 'IMDb ID', 'Movie', 'Year', 'IMDb Score', 'Sebastian', 'Jon', 'Dazraf', 'Average', 'Notes']]
print("Number of Movies:", data.shape[0])
data = data[data.Sebastian.notnull()]
data.head(3)

Number of Movies: 93


,Month,Pick,IMDb ID,Movie,Year,IMDb Score,Sebastian,Jon,Dazraf,Average,Notes
0,3/2019,Farzad,947798,Black Swan,2010,8.0,8.0,9.0,8.0,8.3,Inagural movie!
1,4/2019,Jon,156887,Perfect Blue,1997,8.0,9.0,9.0,7.0,8.3,"Black Swan, but make it anime."
2,5/2019,Steven,311113,Master and Commander: The Far Side of the World,2003,7.4,9.5,8.0,8.0,8.5,https://www.youtube.com/watch?v=mq1YthGFjRI


In [16]:
title_list = []
year_list = []
rating_list = []
plot_outline_list = []
plot_outline = []
synopsis_list = []

for index, row in data.iterrows():
    movie = ia.get_movie(row['IMDb ID'])
    
    title_list.append(movie.get('title'))
    year_list.append(movie.get('year'))
    rating_list.append(movie.get('rating'))
    plot_outline_list.append(text_preprocessing(movie.get('plot outline')))
    plot_outline.append(text_preprocessing(movie.get('plot outline')))
    synopsis_list.append(text_preprocessing(movie.get('synopsis')))
    # movie_data['imdbid'] = movie.get('imdbid')
    # movie_data['directors'] = movie.get('directors') #need to parse this before we can use it
    # movie_data['genres'] = movie.get('genres')
    # movie_data['runtimes'] = movie.get('runtimes')
    # movie_data['budget'] = movie.get('budget')
    # movie_data['gross'] = movie.get('gross')
    # movie_data['synopsis'] = movie.get('synopsis')

    # temp.append([title, year, rating], ignore_index=True)

data['title'] = title_list
data['year'] = year_list
data['rating'] = rating_list
data['plot_outline'] = plot_outline_list
data['plot'] = plot_outline
data['synopsis'] = synopsis_list

TypeError: 'module' object is not callable

In [11]:
#write file out to csv
output_data(data, 'movie_list_current_clean.csv')

Data output to /Users/jonat/Desktop/code/movie_analysis/data/movie_list_current_clean.csv
